In [6]:
ls

Guy_11_Compiled_filtered.xlsx  merged.ipynb
Guy_11_Compiled_updated.xlsx   PFAM.xlsx
Interproscan_matched.xlsx      Protein_accession_final.xlsx


In [ ]:
import pandas as pd

# Load both Excel files
guy_11_df = pd.read_excel('Guy_11_Compiled.xlsx')
protein_df = pd.read_excel('Protein_accession_final.xlsx')

# Merge the dataframes based on matching columns (Gene_ID and Protein_accession)
merged_df = pd.merge(guy_11_df, protein_df[['Protein_accession', 'PFAM Code']], 
                     left_on='Gene_ID', right_on='Protein_accession', 
                     how='left')

# Drop the 'Protein_accession' column from the merged dataframe as it's not needed
merged_df = merged_df.drop(columns=['Protein_accession'])

# Rename 'PFAM Code' to 'matched' for clarity
merged_df = merged_df.rename(columns={'PFAM Code': 'matched'})

# Save the updated dataframe to a new Excel file
merged_df.to_excel('Guy_11_Compiled_updated.xlsx', index=False)

print("Process completed. The updated file is saved as 'Guy_11_Compiled_updated.xlsx'.")


In [10]:
import pandas as pd

# Load the Guy_11_Compiled_updated.xlsx and PFAM.xlsx files
guy_11_df = pd.read_excel('Guy_11_Compiled_updated.xlsx')
pfam_df = pd.read_excel('PFAM.xlsx')

# Strip any whitespace from column names to ensure correct referencing
pfam_df.columns = pfam_df.columns.str.strip()

# Check if 'MEROPS Family' and 'MEROPS Subfamily' exist in PFAM.xlsx
if 'MEROPS Family' not in pfam_df.columns or 'MEROPS Subfamily' not in pfam_df.columns:
    print("Columns 'MEROPS Family' or 'MEROPS Subfamily' not found in PFAM.xlsx!")
else:
    #Create a list of unique elements from the 'matched' column
    unique_matched_list = guy_11_df['matched'].dropna().unique()

    #Initialize an empty list to store the 'Family' information for each row
    family_list = []

    #Iterate over each row in the Guy_11 dataframe to search for matches
    for index, row in guy_11_df.iterrows():
        matched_value = row['matched']
        family_info = []

        # Check if the matched value exists in any of the PFAM columns (PFAM1 to PFAM11)
        for pfam_col in [f'PFAM{i}' for i in range(1, 12)]:
            if matched_value in pfam_df[pfam_col].values:
                # If match is found, retrieve the corresponding 'MEROPS Family' and 'MEROPS Subfamily'
                merops_family = pfam_df.loc[pfam_df[pfam_col] == matched_value, 'MEROPS Family'].values
                merops_subfamily = pfam_df.loc[pfam_df[pfam_col] == matched_value, 'MEROPS Subfamily'].values
                
                #Check if the arrays are not empty
                if merops_family.size > 0 and merops_subfamily.size > 0:
                    family_info.append(f'{merops_family[0]}/{merops_subfamily[0]}')
                elif merops_family.size > 0:
                    family_info.append(merops_family[0])
                elif merops_subfamily.size > 0:
                    family_info.append(merops_subfamily[0])

        # If no match is found, leave the family_info empty (or append 'No match')
        if not family_info:
            family_list.append('No match')
        else:
            # Join all matches found (in case there are multiple) into one string
            family_list.append('; '.join(family_info))

    #Append the 'Family' information to the Guy_11 dataframe
    guy_11_df['Family'] = family_list

    #Export the updated dataframe to a new Excel file
    guy_11_df.to_excel('Guy_11_Compiled_updated_with_Family.xlsx', index=False)

    print("Process completed. The updated file is saved as 'Guy_11_Compiled_updated_with_Family.xlsx'.")


Process completed. The updated file is saved as 'Guy_11_Compiled_updated_with_Family.xlsx'.
